<font size=5> Social distancing: evidence of privilege in a pandemic from smartphones </font><br>
Nabarun Dasgupta, MPH, PhD | nab@unc.edu | @nabarund<br>
*Dr. Dasgupta is an epidemiologist at the University of North Carolina in Chapel Hill. He studies population level patterns of infectious disease, medication safety, and opioids.*<br>
Thanks to Ben White for data munging help. Code available on [GitHub](https://github.com/opioiddatalab/covid).<br>

**Co-authors**<br>
Michele Jonsson Funk, PhD<br>
Allison Lazard, PhD<br>
Benjamin Eugene White<br>
Steve W. Marshall, PhD<br>

<div class="alert alert-warning">

**Warning:** Code and output provided for evaluation only. Paper under review. Results have not been peer-reviewed and should not be used for clinical or public health interpretation.

</div>

On March 23, 2020 Stuart Thompson and Yaryna Serkez of *The New York Times* [published](https://www.nytimes.com/interactive/2020/03/23/opinion/coronavirus-economy-recession.html) a fascinating use of cell phone GPS signal information to gauge movement and commuting, during the advent of social distancing. They compared the state-level data in a [slick graphic](https://www.nytimes.com/interactive/2020/03/23/opinion/coronavirus-economy-recession.html) to political leanings. But we wanted to understand more about other community level characteristics of slow versus fast adopters.<br>
<br>
We were provided access to the same location dataset on social distancing published today in the. We used a data merging approach we have [previously published](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6339867/). Repurposing code from an ongoing project, we merged in community-level data from the Robert Wood Johnson Foundation's [County Health Rankings](https://www.countyhealthrankings.org/). This very rich dataset contains dozens of explanatory variables about health, social, and economic indicators.<br>

---

In [17]:
display "Notebook generated on $S_DATE at $S_TIME ET"

Notebook generated on 21 Apr 2020 at 19:11:06 ET


# Models

In [18]:
// Load pre-procesed data
clear all
set scheme economist
use "https://github.com/opioiddatalab/covid/blob/master/analysiset.dta?raw=true"

// Create results frame
frame create results str20 strat level avg LL UL

// Set up negative binomial model
program define modelrun, rclass
    version 16
    syntax varlist(numeric)
    
    frame change default
    foreach var of local varlist {
    
    di "----- RURALITY-ADJUSTED NEGBIN MODEL -----"
       nbreg `var' levels* homeorder i.rucc, irr nocons vce(r)

    * Store results
        frame post results ("`var'") (1) (round((r(table)[1,1]),.1)) (round((r(table)[5,1]),.1)) (round((r(table)[6,1]),.1))
        frame post results ("`var'") (2) (round((r(table)[1,2]),.1)) (round((r(table)[5,2]),.1)) (round((r(table)[6,2]),.1))
        frame post results ("`var'") (3) (round((r(table)[1,3]),.1)) (round((r(table)[5,3]),.1)) (round((r(table)[6,3]),.1))
        frame post results ("`var'") (4) (round((r(table)[1,4]),.1)) (round((r(table)[5,4]),.1)) (round((r(table)[6,4]),.1))
        frame post results ("`var'") (5) (round((r(table)[1,5]),.1)) (round((r(table)[5,5]),.1)) (round((r(table)[6,5]),.1))
        
        di "Compare to tabular data:"
        table iso5, c(count `var' mean `var' sem `var') 

    di "----- PERCENT DIFFERENCE MODEL -----"
    * Rate difference models
        nbreg `var' levels* homeorder i.rucc, irr vce(r)   
   
   * Plot graph
       frame change results
       la var level "Social Distancing: Lowest (1) to Highest (5)"
       line avg level if inlist(strat,"`var'"), title("`var'")

    frame change default
    
    }
end

// Set up scaled Poisson model

program define modelpoisson, rclass
    version 16
    syntax varlist(numeric)
    
    frame change default
    foreach var of local varlist {

    di "----- RURALITY-ADJUSTED POISSON MODEL -----"
       glm `var' levels* homeorder i.rucc, family(poisson) link(log) scale(x2) eform nocons 

    * Store results
        frame post results ("`var'") (1) (round((r(table)[1,1]),.1)) (round((r(table)[5,1]),.1)) (round((r(table)[6,1]),.1))
        frame post results ("`var'") (2) (round((r(table)[1,2]),.1)) (round((r(table)[5,2]),.1)) (round((r(table)[6,2]),.1))
        frame post results ("`var'") (3) (round((r(table)[1,3]),.1)) (round((r(table)[5,3]),.1)) (round((r(table)[6,3]),.1))
        frame post results ("`var'") (4) (round((r(table)[1,4]),.1)) (round((r(table)[5,4]),.1)) (round((r(table)[6,4]),.1))
        frame post results ("`var'") (5) (round((r(table)[1,5]),.1)) (round((r(table)[5,5]),.1)) (round((r(table)[6,5]),.1))
        
        di "Compare to tabular data:"
        table iso5, c(count `var' mean `var' sem `var') 

    di "----- PERCENT DIFFERENCE MODEL -----"
    * Rate difference models
       glm `var' levels* homeorder i.rucc, family(poisson) link(log) scale(x2) eform 
   
   * Plot graph
       frame change results
       la var level "Social Distancing: Lowest (1) to Highest (5)"
       line avg level if inlist(strat,"`var'"), title("`var'")

    frame change default
    
    }
end

---

# Descriptive Results

In [19]:
// Basic distributions of counties and traces
* iso5 is the main outcome variable representing quintiles of mobility change

tab rucc, m
tab iso5, m
table iso5, c(sum last3_sample)
su last3_sample
di "Total 3-day mobile traces: " r(sum)

* US average change in mobility
qui: su last3_index
    return list

* Mean mobility change by quintile
table iso5, c(count last3_index mean last3_index sem last3_index)

* Mobility quintile boundaries 
bysort iso5: su last3_index

* Rurality/urbanicity by quintile
table iso5, c(median rucc)



  RUCC_2013 |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        342       16.13       16.13
          2 |        308       14.53       30.66
          3 |        277       13.07       43.73
          4 |        167        7.88       51.60
          5 |         79        3.73       55.33
          6 |        445       20.99       76.32
          7 |        318       15.00       91.32
          8 |         95        4.48       95.80
          9 |         89        4.20      100.00
------------+-----------------------------------
      Total |      2,120      100.00


Distancing: |
 Lowest (1) |
 to Highest |
        (5) |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        414       19.53       19.53
          2 |        426       20.09       39.62
          3 |        426       20.09       59.72
          4 |        426       20.09       79.81
          5 |        428       20.1

---
# Healthcare

To gauge overall baseline healthcare access and utilization, we examined primary care providers per 100,000 population and percent uninsured under age 65 (e.g., Medicare eligibility). As a marker for a closely related preventive health behavior, we examined whether earlier influenza vaccination rates were associated with how much the county was likely to slow down in the current coronavirus outbreak. This was quantified as the percent of annual Medicare enrollees having an annual influenza vaccination.

## Primary care providers

We wanted to see if places with more social distancing had better healthcare resources. So we looked at primary care providers per 100,000 population. 


In [20]:
modelrun pcp_rate

----- RURALITY-ADJUSTED NEGBIN MODEL -----
variable homeorder not found


r(111);


### Interpretation
<br><br>

`   pcp_rate |        IRR    [95% Conf. Interval]
-------------+-----------------------------------
     levels1 |   50.45689    46.44897    54.81064
     levels2 |   52.73648    48.70426    57.10253
     levels3 |   55.07755    50.95751    59.53072
     levels4 |   59.75643     55.5741    64.25349
     levels5 |   72.45264    67.53468    77.72872`
     
     
`levels1 |   .6964121   .0276086    -9.13   0.000     .6443491    .7526818`


## Percent uninsured
Percent of without health insurance below Medicare elgibility (age 65).

In [ ]:
// Comparing percent uninsured to social distancing
modelpoisson uninsured_p 

Graph above: Comparing **percent uninsured** to social distancing

Counties with lower social distancing also had a higher proportion of uninsured residents.

## Flu Vaccination

We had a hypothesis that counties that were more involved in preventative behaviors would be more likely to self-isolate more thoroughly. To test this, we examined whether earlier flu vaccination rates impacted how much the county was likely to slow down in the current coronavirus outbreak. This is quantified as the percent of annual Medicare enrollees having an annual flu vaccination, as reported by the Robert Wood Johnson Foundation. Since the flu vaccine is free to all Medicare beneficiaries, and this is the elderly age group with the most influenza mortality, this is a convenient metric to test *a priori* how conscientious the population was, on average.

In [ ]:
// Basic descriptive on background influenza vaccine
frame change default
summ fluvaccine, d
hist fluvaccine, bin(10)

In [ ]:
// Comparing background flu vaccination with current social distancing
modelpoisson fluvaccine

Grpah above: Comparing background **flu vaccination** with current social distancing<br>
`levels1 |   .9409152   .0130748    -4.38   0.000      .915635    .9668934`

--- 
# Economic

There is a trend emerging. So, since the places with more social distancing seem to have more health resources, perhaps there are trends in financial means? We explored two baseline economic metrics in relation to social distancing, one representing the overall wealth of the community and one proxy for poverty: 80th percentile of annual household income in dollars and the percent of school-age children eligible for subsidized or free lunches.

## Household income

In [ ]:
// Comparing 80th percentile income to social distancing
modelrun income80

Graph above: Comparing **80th percentile income** to social distancing

## Subsidized lunches

In [ ]:
modelrun schoollunch

interpretation

---
# Structural
Three lifestyle metrics were selected to provide a diverse snapshot of baseline structural factors that could influence defiance of prolonged stay-at-home orders. The percent of people experiencing food insecurity was derived from Map the Meal Gap project, based on responses from the Current Population Survey and a cost-of-food index. Access to exercise opportunities was the percent of population with adequate access to locations for physical activity. The percent of households with overcrowding was based on the Comprehensive Housing Affordability Strategy measurements.

## Food insecurity

In [ ]:
modelpoisson foodinsec

Interpretation

## Exercise opportunities

In [ ]:
modelrun exercise

Interpretation

In [ ]:
modelpoisson overcrowding

---

# Exploratory analyses

In [ ]:
frame change default
foreach var of varlist drivealone_p {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (mean) `var', by(iso5)
                    la var `var' "% of Drivers"
                        line `var' iso5, note("Commuting Alone by Vehicle")  
                            frame change default
                                frame drop `var'
}

---


In [ ]:
frame change default
foreach var of varlist rucc {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (median) `var', by(iso5)
                    la var `var' "Meidan RUCC"
                        line `var' iso5, note("Urban-Rural")   
                            frame change default
                                frame drop `var'
}

In [ ]:
frame change default
foreach var of varlist longcommute_p {
    table iso5, c(count `var' mean `var' sem `var')
        frame put `var' iso5, into(`var')
            frame change `var'
                collapse (mean) `var', by(iso5)
                    la var `var' "% of Solo Commuters Driving 30+ mins"
                        line `var' iso5, note("Long Solo Commute")     
                            frame change default
                                frame drop `var'
}

---

# Google data